In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import Optional
import time
import math
from typing import Dict, List, Mapping, Optional, Union
import os
import platform

from pororo.tasks.utils.download_utils import download_or_load
import torch
from fairseq.models.roberta import RobertaHubInterface, RobertaModel
from fairseq import hub_utils
import torch.nn.functional as F

import get_functions as gf

In [ ]:
df = gf.load_input()

pre_model = gf.load_model()

batch_token = gf.batch_data_preparation(df, pre_model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

_ = pre_model.to(device)
_ = pre_model.eval()

predicted = gf.get_predicted_label(batch_token, 0, 10, pre_model)

-----DATA LOADED-----
-----MODEL LOADED-----
-----BATCH MAKING DONE-----
-----BATCH DATA PREPARED-----
--- This Process might take SOME time ---


In [5]:
predicted = pd.read_csv('changed_df.txt', delimiter= '\t')
predicted

,Unnamed: 0,text,predicted,changed
0,0,현대차 005380 와 기아 000270 의 주가가 쌩쌩 달리고 있다,"[8, 1, 1, 9, 3, 3, 3, 3, 3, 0, 8, 1, 9, 3, 3, ...","[8, 1, 1, 9, 3, 3, 3, 3, 3, 0, 8, 1, 9, 3, 3, ..."
1,1,2분기 어닝 서프라이즈에 이어 하반기에도 자동차 대기 수요가 지속됨에 따라 호실적이...,"[12, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 4...","[12, 4, 4, 19, 14, 14, 14, 14, 14, 14, 0, 0, 0..."
2,2,증권가에선 목표주가를 상향 조정하며 주가 추가 상승에 힘을 싣고 있다,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,2분기 사상 최대 실적 우호적 환경 지속 전망 4일 마켓포인트에 따르면 현대차 주가...,"[12, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[12, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,4,현대차 주가는 7월1일 18만원에서 이날 19만6500원까지 오름세를 기록하며 9....,"[8, 1, 1, 0, 0, 0, 12, 4, 4, 4, 9, 3, 3, 3, 0,...","[8, 1, 1, 0, 0, 0, 12, 4, 4, 4, 9, 3, 3, 3, 0,..."
...,...,...,...,...
3459,3459,류제현 연구원은 올해 3분기 영업이익은 2068억원으로 비수기에도 이익 호조를 이어...,"[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0...","[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0..."
3460,3460,이어 MR 탱커 시황은 2020년 급등 현상이 재현돼 호황기가 연장되고 컨테이너는 ...,"[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4...","[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4..."
3461,3461,롯데칠성 연간 가이던스 소폭 상향 -하나證 롯데칠성 목표가 24만원 유지 전일종가 ...,"[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3462,3462,하나증권 심은주 권우정 애널리스트가 작성한 롯데칠성 리포트 주요내용은 아래와 같다,"[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2...","[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2..."


In [5]:
predicted = pd.read_csv('changed_df.txt')
predicted = predicted.drop(['Unnamed: 0'], axis = 1)
predicted['predicted'] = predicted['predicted'].apply(lambda k: eval(''.join(k)))

In [7]:
predicted = predicted.drop(['changed', 'clear_predicted', 'clear_changed'], axis = 1)
predicted

,text,predicted
0,현대차 005380 와 기아 000270 의 주가가 쌩쌩 달리고 있다,"[8, 1, 1, 9, 3, 3, 3, 3, 3, 0, 8, 1, 9, 3, 3, ..."
1,2분기 어닝 서프라이즈에 이어 하반기에도 자동차 대기 수요가 지속됨에 따라 호실적이...,"[12, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 4..."
2,증권가에선 목표주가를 상향 조정하며 주가 추가 상승에 힘을 싣고 있다,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,2분기 사상 최대 실적 우호적 환경 지속 전망 4일 마켓포인트에 따르면 현대차 주가...,"[12, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,현대차 주가는 7월1일 18만원에서 이날 19만6500원까지 오름세를 기록하며 9....,"[8, 1, 1, 0, 0, 0, 12, 4, 4, 4, 9, 3, 3, 3, 0,..."
...,...,...
3458,류제현 연구원은 올해 3분기 영업이익은 2068억원으로 비수기에도 이익 호조를 이어...,"[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0..."
3459,이어 MR 탱커 시황은 2020년 급등 현상이 재현돼 호황기가 연장되고 컨테이너는 ...,"[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4..."
3460,롯데칠성 연간 가이던스 소폭 상향 -하나證 롯데칠성 목표가 24만원 유지 전일종가 ...,"[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3461,하나증권 심은주 권우정 애널리스트가 작성한 롯데칠성 리포트 주요내용은 아래와 같다,"[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2..."


In [6]:
def clear_predict(predicted):
    pred_eve = [i for i, e in enumerate(predicted) if e == 19]
    repl = [0] * len(pred_eve)
    
    for pred_eves, repls in zip(pred_eve, repl):
        predicted[pred_eves] = 0
        
    pred_eve2 = [i for i, e in enumerate(predicted) if e == 14]
    repl2 = [0] * len(pred_eve2)
    
    for pred_eves, repls in zip(pred_eve2, repl2):
        predicted[pred_eves] = 0
        
    pred_eve3 = [i for i, e in enumerate(predicted) if e == 16]
    repl3 = [0] * len(pred_eve3) 
    
    for pred_eves, repls in zip(pred_eve3, repl3):
        predicted[pred_eves] = 0
    
    pred_eve4 = [i for i, e in enumerate(predicted) if e == 10]
    repl4 = [0] * len(pred_eve4)   

    for pred_eves, repls in zip(pred_eve4, repl4):
        predicted[pred_eves] = 0
    
    return predicted

In [9]:
removed = []
for j in range(len(predicted)):
    removing = clear_predict(list(predicted['predicted'][j]))
    removed.append(removing)
    
predicted['clear_predicted'] = removed

predicted

,text,predicted,clear_predicted
0,현대차 005380 와 기아 000270 의 주가가 쌩쌩 달리고 있다,"[8, 1, 1, 9, 3, 3, 3, 3, 3, 0, 8, 1, 9, 3, 3, ...","[8, 1, 1, 9, 3, 3, 3, 3, 3, 0, 8, 1, 9, 3, 3, ..."
1,2분기 어닝 서프라이즈에 이어 하반기에도 자동차 대기 수요가 지속됨에 따라 호실적이...,"[12, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 4...","[12, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 4..."
2,증권가에선 목표주가를 상향 조정하며 주가 추가 상승에 힘을 싣고 있다,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,2분기 사상 최대 실적 우호적 환경 지속 전망 4일 마켓포인트에 따르면 현대차 주가...,"[12, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[12, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,현대차 주가는 7월1일 18만원에서 이날 19만6500원까지 오름세를 기록하며 9....,"[8, 1, 1, 0, 0, 0, 12, 4, 4, 4, 9, 3, 3, 3, 0,...","[8, 1, 1, 0, 0, 0, 12, 4, 4, 4, 9, 3, 3, 3, 0,..."
...,...,...,...
3458,류제현 연구원은 올해 3분기 영업이익은 2068억원으로 비수기에도 이익 호조를 이어...,"[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0...","[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0..."
3459,이어 MR 탱커 시황은 2020년 급등 현상이 재현돼 호황기가 연장되고 컨테이너는 ...,"[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 4, 4, 4, 4, 0,..."
3460,롯데칠성 연간 가이던스 소폭 상향 -하나證 롯데칠성 목표가 24만원 유지 전일종가 ...,"[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3461,하나증권 심은주 권우정 애널리스트가 작성한 롯데칠성 리포트 주요내용은 아래와 같다,"[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2...","[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2..."


In [ ]:
def find_all(str_, sub):
    start = 0
    while True:
        str_ = str_.replace(" ", "")
        start = str_.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)
        
def change_events(start_symbol, end_symbol, origin, predicted):
    '''
    Args:
        start_symbol : 이벤트/제품 라벨링 phrase의 시작 기호 
        end_symbol : 이벤트/제품 라벨링 phrase의 마지막 기호
    
    Output:
        모델이 예측한 태그 중 이벤트/제품 라벨링 되어야 할 자리에 이벤트/제품 태그로 replace
    '''
    locs = [list(find_all(origin, start_symbol)), list(find_all(origin, end_symbol))]
    events = []
    for i in range(len(locs[0])):
        indexes1 = locs[0]
        indexes2 = locs[1]
        predicted2 = predicted
        predicted2[indexes1[i] + (-4)*i] = 19
        predicted2[(indexes1[i] + (-4)*i + 1) : (indexes2[i] -3 + (-4)*i + 1)] = [14] * (indexes2[i] - indexes1[i] -3)
        predicted = predicted2
        
    return predicted

def change_products(start_symbol, end_symbol, origin, predicted):
    '''
    Args:
        start_symbol : 이벤트/제품 라벨링 phrase의 시작 기호
        end_symbol : 이벤트/제품 라벨링 phrase의 마지막 기호
    
    Output:
        모델이 예측한 태그 중 이벤트/제품 라벨링 되어야 할 자리에 이벤트/제품 태그로 replace
    '''
    locs = [list(find_all(origin, start_symbol)), list(find_all(origin, end_symbol))]
    events = []
    for i in range(len(locs[0])):
        indexes1 = locs[0]
        indexes2 = locs[1]
        predicted2 = predicted
        predicted2[indexes1[i] + (-4)*i] = 16
        predicted2[(indexes1[i] + (-4)*i + 1) : (indexes2[i] -3 + (-4)*i + 1)] = [10] * (indexes2[i] - indexes1[i] -3)
        predicted = predicted2
        
    return predicted

In [ ]:
removed = []
df2['1'] = df2['1'].apply(lambda k: eval(''.join(k)))
for j in range(len(df2['changed'])):
    removing = clear_predict(list(df2['1'][j]))
    removed.append(removing)
    
df2['clear_predicted'] = removed

df2

In [57]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import Optional
import time
import math
from typing import Dict, List, Mapping, Optional, Union
import os
import platform

from pororo.tasks.utils.download_utils import download_or_load
import torch
from fairseq.models.roberta import RobertaHubInterface, RobertaModel
from fairseq import hub_utils
import torch.nn.functional as F

In [ ]:
def load_model():
    task='ner'
    lang: str = "ko"
    model: Optional[str] = "charbert.base.ko.ner"
    model_name: str = f"bert/{model}"
    kwargs = {}

    ckpt_dir = download_or_load(model_name, lang)

    x = hub_utils.from_pretrained(
        ckpt_dir,
        "model.pt",
        ckpt_dir,
        **kwargs,
    )

    model = RobertaHubInterface(
        x["args"],
        x["task"],
        x["models"][0],
    )
    
    return model
    print('-----MODEL LOADED-----')

In [ ]:
def tokenizer(sent, add_special_tokens=True):
    # tokenize
    x = sent
    x = x.strip()

    if len(x) == 0:
        result = ""
    else:
        # x = [c for c in re.sub("\s+", " ", x)]
        x = list(x)

        result = list()
        for i in range(len(x)):
            if x[i] == " ":
                x[i + 1] = f"▁{x[i+1]}"
                continue # 
            else:
                result.append(x[i])
        result[0] = f"▁{result[0]}"
        result = " ".join(result)
        bpe_sentence = result+

    if add_special_tokens:
        bpe_sentence = f"<s> {result} </s>"

    # tokens in number code
    tokens = model.task.source_dictionary.encode_line(
        bpe_sentence,
        append_eos=False,
        add_if_not_exist=False,
    )
    tokens = tokens[:512]
    tokens = tokens.long()
    
    return result, tokens

def batch_data_preparation(dataframe):
    train_x = []
    for i in range(len(dataframe[0])):
        xs = np.array(tokenizer(''.join(df[0][i])), dtype='object').T[1]
        train_x.append(xs)
    max_token_len = max(list(map(len, train_x)))
    list_token_pad = list(map(lambda token: torch.nn.functional.pad(token, (0, max_token_len - len(token)), value=1), train_x))
    batch_token = torch.stack(list_token_pad)
    
    return batch_token
    print('-----BATCH DATA PREPARED-----')

In [4]:
a = [1, 2, 3, 4, 5]
dff = pd.DataFrame(a)
dff

,0
0,1
1,2
2,3
3,4
4,5


In [6]:
b = a[:]

b = [1, 2, 3, 4, 4]
dff['test'] = b
dff


,0,test
0,1,1
1,2,2
2,3,3
3,4,4
4,5,4


In [68]:
folder = 'Data'

file_name = f'dfs.txt'
file_path = os.path.join(folder, file_name)
df = pd.read_csv(file_path, delimiter = '\t', header = None)
df = df.loc[1700:3465,]
df

,0
1700,겔론당 단가는 고가의 ArF용 포토레지스트와 비슷한 수준의 하이엔드 제품이다
1701,사용 편의성이 높고 ACL 증착 장비 투자비를 절감할 수 있어 기존 ACL를 사용하...
1702,영창케미칼은 낸드플래시 메모리 단수가 증가하면 이에 비례해 <<SOC>> 사용 공정...
1703,영창케미칼은 개발한 <<낸드플래시 메모리용 SOC>>와 관련해 국내외 경쟁사 대비 ...
1704,영창케미칼 관계자는 당사는 국내외 경쟁사 대비 높은 품질을 구현하고 메탈하드마스크 ...
...,...
3460,류제현 연구원은 올해 3분기 영업이익은 2068억원으로 비수기에도 이익 호조를 이어...
3461,이어 MR 탱커 시황은 2020년 급등 현상이 재현돼 호황기가 연장되고 컨테이너는 ...
3462,롯데칠성 연간 가이던스 소폭 상향 -하나證 롯데칠성 목표가 24만원 유지 전일종가 ...
3463,하나증권 심은주 권우정 애널리스트가 작성한 롯데칠성 리포트 주요내용은 아래와 같다


In [66]:
folder = 'Data'

file_name = f'changed_label_1700.csv'
file_path = os.path.join(folder, file_name)
df2 = pd.read_csv(file_path)
df2

,Unnamed: 0,0,1,changed
0,0,겔론당 단가는 고가의 ArF용 포토레지스트와 비슷한 수준의 하이엔드 제품이다,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 0, 0..."
1,1,사용 편의성이 높고 ACL 증착 장비 투자비를 절감할 수 있어 기존 ACL를 사용하...,"[0, 0, 0, 0, 0, 0, 0, 0, 27, 23, 23, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 27, 23, 23, 0, 0, 0, ..."
2,2,영창케미칼은 낸드플래시 메모리 단수가 증가하면 이에 비례해 SOC 사용 공정 수도 ...,"[8, 1, 1, 1, 1, 0, 17, 15, 15, 15, 15, 15, 15,...","[8, 1, 1, 1, 1, 0, 17, 15, 15, 15, 15, 15, 15,..."
3,3,영창케미칼은 개발한 낸드플래시 메모리용 SOC와 관련해 국내외 경쟁사 대비 평탄화 ...,"[8, 1, 1, 1, 1, 0, 0, 0, 0, 17, 15, 15, 15, 15...","[8, 1, 1, 1, 1, 0, 0, 0, 0, 16, 10, 10, 10, 10..."
4,4,영창케미칼 관계자는 당사는 국내외 경쟁사 대비 높은 품질을 구현하고 메탈하드마스크 ...,"[8, 1, 1, 1, 1, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...","[8, 1, 1, 1, 1, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
1760,1760,류제현 연구원은 올해 3분기 영업이익은 2068억원으로 비수기에도 이익 호조를 이어...,"[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0...","[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0..."
1761,1761,이어 MR 탱커 시황은 2020년 급등 현상이 재현돼 호황기가 연장되고 컨테이너는 ...,"[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4...","[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4..."
1762,1762,롯데칠성 연간 가이던스 소폭 상향 -하나證 롯데칠성 목표가 24만원 유지 전일종가 ...,"[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1763,1763,하나증권 심은주 권우정 애널리스트가 작성한 롯데칠성 리포트 주요내용은 아래와 같다,"[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2...","[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2..."


In [69]:
df2['length'] = df2['0'].str.len()
df2 = df2[df2.length <=512]
df['length'] = df[0].str.len()
df = df[df.length <=512]
df2 = df2.reset_index(drop = True)
df = df.reset_index(drop = True)

In [70]:
def find_all(str_, sub):
    start = 0
    while True:
        str_ = str_.replace(" ", "")
        start = str_.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)
        
def change_events(start_symbol, end_symbol, origin, predicted):
    '''
    Args:
        start_symbol : 이벤트/제품 라벨링 phrase의 시작 기호
        end_symbol : 이벤트/제품 라벨링 phrase의 마지막 기호
    
    Output:
        모델이 예측한 태그 중 이벤트/제품 라벨링 되어야 할 자리에 이벤트/제품 태그로 replace
    '''
    locs = [list(find_all(origin, start_symbol)), list(find_all(origin, end_symbol))]
    events = []
    for i in range(len(locs[0])):
        indexes1 = locs[0]
        indexes2 = locs[1]
        predicted2 = predicted
        predicted2[indexes1[i] + (-4)*i] = 19
        predicted2[(indexes1[i] + (-4)*i + 1) : (indexes2[i] -3 + (-4)*i + 1)] = [14] * (indexes2[i] - indexes1[i] -3)
        predicted = predicted2
        
    return predicted

def change_products(start_symbol, end_symbol, origin, predicted):
    '''
    Args:
        start_symbol : 이벤트/제품 라벨링 phrase의 시작 기호
        end_symbol : 이벤트/제품 라벨링 phrase의 마지막 기호
    
    Output:
        모델이 예측한 태그 중 이벤트/제품 라벨링 되어야 할 자리에 이벤트/제품 태그로 replace
    '''
    locs = [list(find_all(origin, start_symbol)), list(find_all(origin, end_symbol))]
    events = []
    for i in range(len(locs[0])):
        indexes1 = locs[0]
        indexes2 = locs[1]
        predicted2 = predicted
        predicted2[indexes1[i] + (-4)*i] = 16
        predicted2[(indexes1[i] + (-4)*i + 1) : (indexes2[i] -3 + (-4)*i + 1)] = [10] * (indexes2[i] - indexes1[i] -3)
        predicted = predicted2
        
    return predicted

def clear_predict(predicted):
    pred_eve = [i for i, e in enumerate(predicted) if e == 19]
    repl = [0] * len(pred_eve)
    
    for pred_eves, repls in zip(pred_eve, repl):
        predicted[pred_eves] = 0
        
    pred_eve2 = [i for i, e in enumerate(predicted) if e == 14]
    repl2 = [0] * len(pred_eve2)
    
    for pred_eves, repls in zip(pred_eve2, repl2):
        predicted[pred_eves] = 0
        
    pred_eve3 = [i for i, e in enumerate(predicted) if e == 16]
    repl3 = [0] * len(pred_eve3) 
    
    for pred_eves, repls in zip(pred_eve3, repl3):
        predicted[pred_eves] = 0
    
    pred_eve4 = [i for i, e in enumerate(predicted) if e == 10]
    repl4 = [0] * len(pred_eve4)   

    for pred_eves, repls in zip(pred_eve4, repl4):
        predicted[pred_eves] = 0
    
    return predicted

In [71]:
removed = []
df2['1'] = df2['1'].apply(lambda k: eval(''.join(k)))
for j in range(len(df2['changed'])):
    removing = clear_predict(list(df2['1'][j]))
    removed.append(removing)
    
df2['clear_predicted'] = removed

df2

,Unnamed: 0,0,1,changed,length,clear_predicted
0,0,겔론당 단가는 고가의 ArF용 포토레지스트와 비슷한 수준의 하이엔드 제품이다,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 0, 0...",42,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 0, 0..."
1,1,사용 편의성이 높고 ACL 증착 장비 투자비를 절감할 수 있어 기존 ACL를 사용하...,"[0, 0, 0, 0, 0, 0, 0, 0, 27, 23, 23, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 27, 23, 23, 0, 0, 0, ...",68,"[0, 0, 0, 0, 0, 0, 0, 0, 27, 23, 23, 0, 0, 0, ..."
2,2,영창케미칼은 낸드플래시 메모리 단수가 증가하면 이에 비례해 SOC 사용 공정 수도 ...,"[8, 1, 1, 1, 1, 0, 17, 15, 15, 15, 15, 15, 15,...","[8, 1, 1, 1, 1, 0, 17, 15, 15, 15, 15, 15, 15,...",105,"[8, 1, 1, 1, 1, 0, 17, 15, 15, 15, 15, 15, 15,..."
3,3,영창케미칼은 개발한 낸드플래시 메모리용 SOC와 관련해 국내외 경쟁사 대비 평탄화 ...,"[8, 1, 1, 1, 1, 0, 0, 0, 0, 17, 15, 15, 15, 15...","[8, 1, 1, 1, 1, 0, 0, 0, 0, 16, 10, 10, 10, 10...",120,"[8, 1, 1, 1, 1, 0, 0, 0, 0, 17, 15, 15, 15, 15..."
4,4,영창케미칼 관계자는 당사는 국내외 경쟁사 대비 높은 품질을 구현하고 메탈하드마스크 ...,"[8, 1, 1, 1, 1, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...","[8, 1, 1, 1, 1, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...",159,"[8, 1, 1, 1, 1, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
1759,1760,류제현 연구원은 올해 3분기 영업이익은 2068억원으로 비수기에도 이익 호조를 이어...,"[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0...","[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0...",152,"[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0..."
1760,1761,이어 MR 탱커 시황은 2020년 급등 현상이 재현돼 호황기가 연장되고 컨테이너는 ...,"[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4...","[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4...",147,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 4, 4, 4, 4, 0,..."
1761,1762,롯데칠성 연간 가이던스 소폭 상향 -하나證 롯데칠성 목표가 24만원 유지 전일종가 ...,"[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",99,"[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1762,1763,하나증권 심은주 권우정 애널리스트가 작성한 롯데칠성 리포트 주요내용은 아래와 같다,"[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2...","[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2...",45,"[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2..."


In [72]:
og_pred = df2['clear_predicted']
og_predd = og_pred[:]
changed = []
for i in range(len(df2['0'])):
    pred = change_events("{{", "}}", df[0][i], og_predd[i])
    changed.append(pred)
    
changed2 = []
for i in range(len(df2['0'])):
    pred2 = change_products("<<", ">>", df[0][i], changed[i])
    changed2.append(pred2)

In [75]:
df2['clear_changed'] = changed2
df2

,Unnamed: 0,0,1,changed,length,clear_predicted,clear_changed
0,0,겔론당 단가는 고가의 ArF용 포토레지스트와 비슷한 수준의 하이엔드 제품이다,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 0, 0...",42,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 0, 0..."
1,1,사용 편의성이 높고 ACL 증착 장비 투자비를 절감할 수 있어 기존 ACL를 사용하...,"[0, 0, 0, 0, 0, 0, 0, 0, 27, 23, 23, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 27, 23, 23, 0, 0, 0, ...",68,"[0, 0, 0, 0, 0, 0, 0, 0, 27, 23, 23, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 27, 23, 23, 0, 0, 0, ..."
2,2,영창케미칼은 낸드플래시 메모리 단수가 증가하면 이에 비례해 SOC 사용 공정 수도 ...,"[8, 1, 1, 1, 1, 0, 17, 15, 15, 15, 15, 15, 15,...","[8, 1, 1, 1, 1, 0, 17, 15, 15, 15, 15, 15, 15,...",105,"[8, 1, 1, 1, 1, 0, 17, 15, 15, 15, 15, 15, 15,...","[8, 1, 1, 1, 1, 0, 17, 15, 15, 15, 15, 15, 15,..."
3,3,영창케미칼은 개발한 낸드플래시 메모리용 SOC와 관련해 국내외 경쟁사 대비 평탄화 ...,"[8, 1, 1, 1, 1, 0, 0, 0, 0, 17, 15, 15, 15, 15...","[8, 1, 1, 1, 1, 0, 0, 0, 0, 16, 10, 10, 10, 10...",120,"[8, 1, 1, 1, 1, 0, 0, 0, 0, 16, 10, 10, 10, 10...","[8, 1, 1, 1, 1, 0, 0, 0, 0, 16, 10, 10, 10, 10..."
4,4,영창케미칼 관계자는 당사는 국내외 경쟁사 대비 높은 품질을 구현하고 메탈하드마스크 ...,"[8, 1, 1, 1, 1, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...","[8, 1, 1, 1, 1, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...",159,"[8, 1, 1, 1, 1, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...","[8, 1, 1, 1, 1, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
1759,1760,류제현 연구원은 올해 3분기 영업이익은 2068억원으로 비수기에도 이익 호조를 이어...,"[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0...","[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0...",152,"[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0...","[11, 6, 6, 5, 2, 2, 0, 12, 4, 4, 4, 4, 0, 0, 0..."
1760,1761,이어 MR 탱커 시황은 2020년 급등 현상이 재현돼 호황기가 연장되고 컨테이너는 ...,"[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4...","[0, 0, 16, 10, 10, 10, 0, 0, 0, 12, 4, 4, 4, 4...",147,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 4, 4, 4, 4, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 4, 4, 4, 4, 0,..."
1761,1762,롯데칠성 연간 가이던스 소폭 상향 -하나證 롯데칠성 목표가 24만원 유지 전일종가 ...,"[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",99,"[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[8, 1, 1, 1, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1762,1763,하나증권 심은주 권우정 애널리스트가 작성한 롯데칠성 리포트 주요내용은 아래와 같다,"[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2...","[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2...",45,"[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2...","[8, 1, 1, 1, 11, 6, 6, 11, 6, 6, 5, 2, 2, 2, 2..."


In [76]:
df2.to_csv('Data/changed_label_1700.csv', encoding = 'utf-8-sig')